In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import json

# I . Scraping : séléctions des collections

## a. import des données via API du site Open CNFT 

In [2]:
urlAT = 'https://api.opencnft.io/1/rank?window=all'
rAT = requests.get(urlAT)

In [3]:
ALLTIME = pd.json_normalize(rAT.json()['ranking'])

In [4]:
url30 = 'https://api.opencnft.io/1/rank?window=30d'
r30 = requests.get(url30)

In [5]:
THIRTY_D = pd.json_normalize(r30.json()['ranking'])

# 

## b. Travail sur les DB pour les cleaners.

### 1 . enlever la colonne 'thumbnail' 

In [6]:
del ALLTIME['thumbnail']

In [7]:
del THIRTY_D['thumbnail']

### 2. Changer la classe de chaque colonne

In [8]:
ALLTIME['policies'] = ALLTIME['policies'].astype('str') 
THIRTY_D['policies'] = THIRTY_D['policies'].astype('str') 

### 3. Enlever les braquettes début/fin de la colonne 'policies'

In [9]:
i = 0 
while i < len(ALLTIME):
    ALLTIME.loc[i, 'policies'] = ALLTIME.policies[i][2:ALLTIME.policies[i].find("']")]
    i+= 1

In [10]:
i = 0 
while i < len(THIRTY_D):
    THIRTY_D.loc[i, 'policies'] = THIRTY_D.policies[i][2:THIRTY_D.policies[i].find("']")]
    i+= 1

## c. Début de la séléction selon des collections

### 1 . Collections qui font mieux sur 30D que les collections qui sont présentes dans le top10 30D et TOP 10 ALL TIME

In [12]:
# peut etre que 

In [13]:
BESTALL = []
i = 0 
while i < len(THIRTY_D[0:11]):
    y = 0
    while y < len(ALLTIME[0:6]): 
        if THIRTY_D.policies[i] == ALLTIME.policies[y]:
            BESTALL.append(THIRTY_D.loc[i])
        y += 1
    i+=1

In [14]:
BESTALL = pd.DataFrame(BESTALL)

In [15]:
BESTALL = BESTALL.reset_index(drop = True)

In [16]:
NEW_TOBUY = []
i = 0 
while i < len(THIRTY_D[0:11]):
    if THIRTY_D.volume[i] > BESTALL.volume.min() and THIRTY_D.policies[i] not in ALLTIME.policies[0:13].values:
        NEW_TOBUY.append(THIRTY_D.loc[i]) 
    i+= 1

In [17]:
NEW_TOBUY = pd.DataFrame(NEW_TOBUY)
NEW_TOBUY = NEW_TOBUY.reset_index(drop = True)

In [18]:
NEW_TOBUY.name = NEW_TOBUY.name.str.replace(' ', '').str.replace('-', '').str.lower()

### 2 . TOP 'N' ALL TIME --> basique recherche d'opportunité 

In [19]:
#regarder si dans le top 20 ALL TIME, il y a des opportunités d'achats. 
# Genre X % moins cheros que le marché. 

In [20]:
#Définition de la plage sur laquelle on veut travailler, 
# m, n la plage des collections sur lesquelles on travaille, NE PAS PRENDRE UN N TROP ELEVE et éviter un range trop élévé. Calcul lourd
# p --> le floor price minimum, histoire de ne pas avoir que des collections qui vallent un bras, NE PAS PRENDRE UN P TROP ELEVE à moins d^'tre dasn el top'

In [21]:
m = 11
n = 20
p = 300

In [22]:
df = ALLTIME[m:n]
ALLTIME_TOP_N_TOCHECK = df[df.floor_price < p]

In [23]:
ALLTIME_TOP_N_TOCHECK = ALLTIME_TOP_N_TOCHECK.reset_index(drop = True)

In [24]:
ALLTIME_TOP_N_TOCHECK.name = ALLTIME_TOP_N_TOCHECK.name.str.replace(' ', '').str.replace('-', '').str.lower()

### 3. TOP15 ALLTIME en chute depuis 7 jours --> opportunités achat possibles

### 4. TOP 10 7 DAYS non présents dans le top 20 ALL TIME & TOP 10 ALL TIME : (à réfléchir)

## d. Extraction des fichiers

In [27]:
ALLTIME_TOP_N_TOCHECK.to_csv(r'C:\Users\marti\Desktop\THP\PF\CSV_STRAT\ALLTIME_TOP_N_TOCHECK.csv')

In [28]:
NEW_TOBUY.to_csv(r'C:\Users\marti\Desktop\THP\PF\CSV_STRAT\NEW_TOBUY.csv')

In [29]:
ALLTIME_TOP_N_TOCHECK

,name,volume,total_tx,total_assets,floor_price,volume_today,volume_ystd,1dChange,volume_week,volume_last_week,7dChange,policies,thumbnail_type,total_minted,total_owners,link
0,aquafarmers,3.710687e+06,6922,4695,220.0,14118.0,2937.0,3.806946,30162.0,16600.00,0.816988,86ec26a91051e4d42df00b023202e177a0027dca4294a2...,image/png,[10001],[3145],https://opencnft.io/p4PovekqpGS5jv29yDl
1,zombiechains,3.527482e+06,10844,6078,90.0,1625.0,200.0,7.125000,8506.0,3972.69,1.141118,96580bbc4fe27ac0d127db3f8a0dc698c58d303d8cae87...,None,[10000],[2668],https://opencnft.io/1JX1lE9OGnHdPoB66aeDiBo
2,cabinsbytheapesociety,3.442645e+06,5670,4343,170.0,4955.0,1216.0,3.074836,24024.0,24967.00,-0.037770,d4e087164acf8314f1203f0b0996f14908e2a199a296d0...,image/gif,[10001],[2940],https://opencnft.io/GP2aEomaPdfmqOX9zVQ8UXqZm1...
3,cornucopiasbubblejettsprinter2022,3.244915e+06,10612,7248,169.0,4430.0,2457.9,0.802352,26275.9,21298.50,0.233697,e282271ec9251ba23fb123b0f53618b35cf5a6cde4170c...,image/png,[12001],[6196],https://opencnft.io/4omZwPwEJ7u3ke5l806eUk1EEB...


In [30]:
NEW_TOBUY

,name,volume,total_tx,total_assets,floor_price,volume_today,volume_ystd,1dChange,volume_week,volume_last_week,7dChange,policies,thumbnail_type,total_minted,total_owners,link
0,themallardorder,3487665.02,5423,3811,322.0,22794.33,8171.00,1.789662,125016.23,216228.44,-0.421833,901ba6e9831b078e131a1cc403d6139af21bda255cea6c...,image/png,[6501],[1561],https://opencnft.io/PJxZ87VVVAsB4NP5Mo7dsL21xqB2D
1,raccoonsyndicate,1133321.90,3780,3054,175.0,8480.00,14776.99,-0.426135,181652.16,951669.74,-0.809123,59142df7dfea56a5b76b842c206fc7bd844b43a2d10f5e...,image/png,[5000],[931],https://opencnft.io/xyn6jxVvoeHvvmLMojx2fzVN463zv
2,thejrney,876078.09,5148,4025,47.0,2189.00,5061.70,-0.567537,36996.33,61591.27,-0.399325,c204b7595ab87f70a4a4d3a6c40833e8942b990bacf492...,image/jpg,[6543],[1375],https://opencnft.io/5vBGJ76a9YI3wNG1
3,virtuacardanoislandland,510728.90,505,465,205.0,3501.00,13714.99,-0.744732,100052.49,122964.40,-0.186330,347f991d4b306919e3e7b01f1cdececa06c2e493a21a0d...,None,[6375],[2035],https://opencnft.io/VNOYqZL5AXSkRZmPYandizo7mx...
